In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
# import random


In [2]:
# os.chdir('/')
print(os.getcwd())

C:\Users\smouz\Google Drive\jupyter_notebook\capstone


In [3]:
os.environ['HOMEPATH']

'\\Users\\smouz'

In [4]:
if 'Capstone' not in os.getcwd():
    os.chdir(str(os.environ['HOMEPATH']) + '\OneDrive\Desktop\DataSci\DataSci_Python\SP-Capstone-PY\Capstone')
print('Working directory:', os.getcwd())

Working directory: C:\Users\smouz\OneDrive\Desktop\DataSci\DataSci_Python\SP-Capstone-PY\Capstone


### Dataset
Import data and count missing values for solar radiation. These values will be filled using data obtained through an API.

In [5]:
pv_df = pd.read_csv('pv_df_short.csv', low_memory=False)
pv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886655 entries, 0 to 886654
Data columns (total 23 columns):
state                          886655 non-null object
date_installed                 886655 non-null object
incentive_prog_names           710174 non-null object
size_kw                        886655 non-null float64
zipcode                        886655 non-null int64
install_type                   886655 non-null object
installer                      641410 non-null object
cost_per_watt                  661381 non-null float64
cost                           661288 non-null float64
city                           712730 non-null object
utility_clean                  706910 non-null object
tech_1                         533924 non-null object
model1_clean                   533924 non-null object
county                         877357 non-null object
annual_pv_prod                 886655 non-null float64
annual_insolation              695447 non-null float64
rebate               

In [6]:
def missing_val_count(df, col_name, show_missing=False):
    '''
    Count missing values and show its index.
    Returns DF if show_missing=True
    '''
    if show_missing == True:
        return df[df.loc[:, col_name].isnull()]
    else:
        return col_name, np.count_nonzero(df.loc[:, col_name].isnull().values)

In [7]:
# format zipcode
pv_df['zipcode'] = pv_df['zipcode'].map('{:05d}'.format)


In [8]:
missing_val_count(pv_df, 'annual_insolation')

('annual_insolation', 191208)

### NREL API
Obtain solar radiation data using API

Only 1,000 requests can be made per hour. At first glance, with 191,208 missing values it would take about 191 hours to get all values through the API. In terms of days, it would take roughly 8 days to complete.

In [9]:
pv_df.groupby('state')['annual_insolation'].mean().sort_values()

state
ct    4.209904
ma    4.232425
oh    4.298287
nj    4.331036
ny    4.351757
pa    4.380561
or    4.404228
wi    4.415797
mn    4.459524
nh    4.473857
ri    4.539976
md    4.593002
de    4.613283
dc    4.618089
mo    4.808802
ar    5.050008
ut    5.071128
fl    5.219673
tx    5.307899
ca    5.468537
nv    6.160320
nm    6.383599
az    6.464277
ak         NaN
al         NaN
co         NaN
ga         NaN
hi         NaN
ia         NaN
id         NaN
il         NaN
in         NaN
ks         NaN
ky         NaN
la         NaN
me         NaN
mi         NaN
ms         NaN
mt         NaN
nc         NaN
nd         NaN
ne         NaN
ok         NaN
pr         NaN
sc         NaN
sd         NaN
tn         NaN
va         NaN
vt         NaN
wa         NaN
wv         NaN
wy         NaN
Name: annual_insolation, dtype: float64

In [10]:
pv_df[pv_df['state'] == 'ca'].annual_insolation.describe()

count    431109.000000
mean          5.468537
std           0.383630
min           1.268869
25%           5.354301
50%           5.487600
75%           5.644894
max           6.761269
Name: annual_insolation, dtype: float64

In [11]:
pv_df['city_state'] = pv_df['city'] + ', ' + pv_df['state']

In [12]:
pv_df['city_state'].nunique()

9584

In [13]:
print(pv_df['zipcode'].nunique())
print(pv_df['county'].nunique())

11340
846


In [14]:
def get_data(baseurl, params=None):
    """ Returns results of a request in JSON format"""
    re = requests.get(baseurl, params)
    try:
        if re.status_code == 200:
            return re.json()
        else:
            return re.json()
    except Exception as e:
        print(e)

In [15]:
pv_df[['county', 'state']].tail(20)

,county,state
886635,NaN,ny
886636,NaN,ny
886637,NaN,ny
886638,NaN,oh
886639,NaN,oh
886640,NaN,oh
886641,NaN,oh
886642,NaN,va
886643,NaN,pa
886644,NaN,pa


### Insolation for each county
For each unique county, we'll obtain the annual insolation value. The only parameter that's really needed in this case is the county so we'll keep all other parameters constant and interate through counties. Also, it is a good idea to implement a time delay between each request to be respectful of the servers. 

In [16]:
# create subset DF where annual_insolation values are missing
pv_subset = pv_df[pv_df['annual_insolation'].isnull()]
print(pv_subset.city_state.nunique())

1826


In [18]:
# combine count and state to create an address for a request
counties_addr = pv_subset['county'] + ', ' + pv_subset['state']
print(counties_addr.nunique())

# create numpy array of unique addreses
counties = counties_addr.unique()
np.count_nonzero(counties_addr.notnull().values)

1069


array(['travis, tx', nan, 'seminole, fl', ..., 'eddy, nm', 'union, nm',
       'modoc, ca'], dtype=object)

In [77]:
# pd.Series(counties).isnull()
# type(counties[1]) == float


In [19]:

# iterate over counties and extract annual insolation
annual_insol = []
address_input = []
API_KEY = 'my_key'
url = 'https://developer.nrel.gov/api/pvwatts/v6.json?'
for address in counties:
    # np.nan is a float, therefore do no use
    if type(address) == str:
        try:
#             print('\nWorking...')
#             print('Requesting address:', address)
            params_in = {'address': address,
                         'system_capacity': 5,
                         'azimuth': 180,
                         'tilt': 40,
                         'tracking': 1,
                         'api_key': API_KEY,
                         'array_type': 1,
                         'module_type': 0,
                         'losses': 10
                         }
            # make request
            res = get_data(url, params=params_in)
            annual_insol.append(res['outputs']['solrad_annual'])
            address_input.append(res['inputs']['address'])
            time.sleep(random.randint(2,6))
        except Exception as e:
            print(e)


Working...
Requesting address: travis, tx
Sleeping 6 seconds

Working...
Requesting address: seminole, fl
Sleeping 6 seconds

Working...
Requesting address: pinal, az
Sleeping 6 seconds

Working...
Requesting address: pima, az
Sleeping 6 seconds

Working...
Requesting address: orange, fl
Sleeping 6 seconds

Working...
Requesting address: dc, dc
Sleeping 6 seconds

Working...
Requesting address: sullivan, ny
Sleeping 6 seconds

Working...
Requesting address: tompkins, ny
Sleeping 6 seconds

Working...
Requesting address: queens, ny
Sleeping 6 seconds

Working...
Requesting address: oneida, ny
Sleeping 6 seconds

Working...
Requesting address: columbia, ny
Sleeping 6 seconds

Working...
Requesting address: montgomery, ny
Sleeping 6 seconds

Working...
Requesting address: warren, ny
Sleeping 6 seconds

Working...
Requesting address: nassau, ny
Sleeping 6 seconds

Working...
Requesting address: suffolk, ny
Sleeping 6 seconds

Working...
Requesting address: boone, mo
Sleeping 6 seconds

Wo

In [20]:
# create dataframe
insolation_df = pd.DataFrame({'county_state': address_input, 'solar_rad': annual_insol})
insolation_df = insolation_df.drop_duplicates()
insolation_df = insolation_df.reset_index(drop=True)


In [21]:
# write file
filename = 'insolation_df.csv'
print('Writing to file:', filename)
insolation_df.to_csv(filename, encoding='utf-8', index=False)
print('Finished writing to:', os.getcwd())

Writing to file: insolation_df.csv
Finished writing to: C:\Users\smouz\OneDrive\Desktop\DataSci\DataSci_Python\SP-Capstone-PY\Capstone
